__This notebook is used for testing the logger.__<br>
Do not grab the logger from this notebook. <br>
Instead import the following:
```py
import os
import sys
logger_path = os.path.join(os.getcwd(), 'logger')
if not logger_path in sys.path:
    sys.path.append(logger_path)
from Logger import Logger
```

In [3]:
import os
import sqlite3
import sys
from datetime import datetime
from enum import Enum

In [40]:
class Logger:
    """
    A component used for logging events of other components.
    Designed to help increase readibility of logs by allowing the user to
    customise their own event messages and specify an urgency.

    Params:
        database_name: str - The name the logger will use for the logs database file (e.g. "logs_scaper.db") .

        working_dir: str - The working directory where the logger will store the log files.
            default value: '..'
    """
    def __init__(self, database_name, working_dir=''):
        assert type(database_name) is str, 'Database name must be of type str'
        assert len(database_name) != 0, 'Database name must be provided to logger'

        assert type(working_dir) is str, 'Working directory must be of type str'

        # Urgency enum entrypoint
        self.urgency = self._Urgency
        # Today's date in the format Mon_DD_MM_YYYY
        self.today = datetime.now(tz=None).strftime('%a_%d_%b_%Y')

        # The working directory for the logger
        if working_dir == '':
            d = os.getcwd()
        # Name of the database
        self.database_name = database_name

        # Database connection
        self._conn = None
        # Database cursor
        self._cursor = None
        self._connect()

    def _connect(self):
        """
        Establishes the connection and cursor to the database specified
        when instantiating the Logger.
        """
        # Change to working directory
        os.chdir(self.working_dir)
        # If the logs directory does not exist, make it
        print(os.path.exists(os.path.join(self.working_dir, 'logs')))
        if not os.path.exists(os.path.join(self.working_dir, 'logs')):
            os.mkdir('logs')
        os.chdir('logs')

        # Establish a database connection if it does not exist
        if self._conn is None:
            self._conn = sqlite3.connect(self.database_name)
        # Create the cursor from the connection
        self._cursor = self._conn.cursor()

    def log(self, message, urgency):
        """
        Log the given message to the database specified when creating
        the Logger. Urgency may be specified but is not required.

        Params:
            message: str - The message to log to the database.

            urgency: Logger._Urgency - The severity of the message.
                default value: Logger._Urgency.NONE
        """
        pass

    def get_timestamp(self):
        """
        Get the current time as a timestamp in the format HH:MM:SS.MS (UTC).

        Returns:
            A string representing the timestamp.
        """
        # Create the timestamp in the format hours:minutes:seconds.microseconds
        timestamp = datetime.now(tz=None).strftime('%H:%M:%S.%f (UTC)')
        print(type(timestamp))

        pass

    class _Urgency(Enum):
        """
        A private enum class of the Logger component.
        Contains urgency levels along with a description of what
        the intended meaning is.
        """
        NONE     = 0 # Default value when urgency is not specified
        LOW      = 1 # Expected event or result
        MODERATE = 2 # Potentially unexpected event or result
        HIGH     = 3 # Caught exceptions, unexpected events
        SEVERE   = 4 # Exceptions that break the system, bugs, etc.

In [41]:
logger = Logger('Test')
logger.get_timestamp()

False


FileExistsError: [Errno 17] File exists: 'logs'

['/home/chris/dev/stock-market-predictor/logger', '/home/chris/anaconda3/lib/python38.zip', '/home/chris/anaconda3/lib/python3.8', '/home/chris/anaconda3/lib/python3.8/lib-dynload', '', '/home/chris/anaconda3/lib/python3.8/site-packages', '/home/chris/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg', '/home/chris/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/home/chris/.ipython']
/home/chris/dev/stock-market-predictor/logger
/home/chris/dev/stock-market-predictor


NameError: name '__file__' is not defined